In [4]:
# Inspecting the uploaded file to understand its header structure
file_path = '78.5_3s.osc'

# Read the first 8 KB to examine the structure and header
with open(file_path, 'rb') as file:
    file_content = file.read(0x1800)  # Read up to offset 0x1760 (inclusive)

file_content[:0x180]  # Displaying the first 384 bytes to inspect header structure


# Extract the full header from the file to offset 0x1760
header_offset = 0x1770  # End of the header section

with open(file_path, 'rb') as file:
    header = file.read(header_offset)

# Save the header to confirm its extraction
header[:64], len(header)  # Displaying the start of the header and its length for verification


# Define the output file path
output_file_path = 'generated_file.osc'

# Define the maximum 16-bit value and pattern size (for a reasonable file size)
max_16bit_value = 0xFFFF
pattern_repeats = 1000  # Number of cycles of counting up and down

pattern = bytearray()
for _ in range(pattern_repeats):
    # Count up from 1 to 0xFFFF
    for i in range(1, max_16bit_value + 1):
        pattern.extend(i.to_bytes(2, byteorder='big'))
    # Count down from 0xFFFE to 1
    for i in range(max_16bit_value - 1, 0, -1):
        pattern.extend(i.to_bytes(2, byteorder='big'))

with open(output_file_path, 'wb') as new_file:
    new_file.write(header)
    new_file.write(pattern)

In [7]:
file1 = open("generated_file - copy.asc", "r")
file2 = open("generated_file.asc", "r")

# Skip headers until "ROI pixel values follow"
while True:
    line1 = file1.readline()
    line2 = file2.readline()
    if "ROI pixel values follow" in line1:
        break

pixel_index = 0
differences = []
line_count = 0

while True:
    line1 = file1.readline()
    line2 = file2.readline()
    if not line1 or not line2:
        break  # End of file
    
    line_count += 1

    values1 = line1.split()
    values2 = line2.split()

    # Compare each value pair
    for i, (v1, v2) in enumerate(zip(values1, values2)):
        pixel_index += 1
        if v1 != v2:
            # Record line number (relative to the first pixel line), position in line (i+1), and pixel index
            differences.append((line_count, i+1, pixel_index, v1, v2))

file1.close()
file2.close()

print("Found", len(differences), "differences.")
for diff in differences:
    line_num, val_pos, pix_idx, orig_val, new_val = diff
    print(f"Line {line_num}, Value {val_pos}, Pixel {pix_idx}: {orig_val} != {new_val}")


Found 0 differences.
